In [0]:
file_location = "/FileStore/tables/customers_refined.csv"
file_type = "csv"

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
df = spark.read.format('delta') \
  .load(file_location)

In [0]:
df.columns

Out[4]: ['row_id',
 'order_id',
 'order_date',
 'order_priority',
 'order_quantity',
 'sales',
 'discount',
 'ship_mode',
 'profit',
 'unit_price',
 'shipping_cost',
 'customer_name',
 'province',
 'region',
 'customer_segment',
 'product_category',
 'product_sub_category',
 'product_name',
 'product_container',
 'ship_date',
 'loyalty_points',
 'loyalty_type',
 'customer_id',
 'total_orders_count',
 'customer_since',
 'total_years']

In [0]:
orders_df = df.select('customer_id', 'order_id', 'order_date', 'ship_date', 'order_priority', 'order_quantity')

In [0]:
orders_df.show(5)

+--------------------+--------+-------------------+----------+--------------+--------------+
|         customer_id|order_id|         order_date| ship_date|order_priority|order_quantity|
+--------------------+--------+-------------------+----------+--------------+--------------+
|001492ac094ba3c98...|   18374|2012-09-30 00:00:00|2012-10-02|           Low|            21|
|001492ac094ba3c98...|   25314|2012-02-24 00:00:00|2012-02-26|          High|            43|
|001492ac094ba3c98...|   31106|2012-02-16 00:00:00|2012-02-16|           Low|            37|
|001492ac094ba3c98...|   31106|2012-02-16 00:00:00|2012-02-21|           Low|             7|
|001492ac094ba3c98...|   31106|2012-02-16 00:00:00|2012-02-18|           Low|            45|
+--------------------+--------+-------------------+----------+--------------+--------------+
only showing top 5 rows



In [0]:
orders_df.filter("ship_date is null").count()

Out[7]: 290

In [0]:
orders_df.filter("ship_date is null").show(4)

+--------------------+--------+-------------------+---------+--------------+--------------+
|         customer_id|order_id|         order_date|ship_date|order_priority|order_quantity|
+--------------------+--------+-------------------+---------+--------------+--------------+
|001492ac094ba3c98...|   43493|2012-05-10 00:00:00|     null|          High|            22|
|0105946ed08fb4ccf...|   48452|2010-10-09 00:00:00|     null|          High|            15|
|018fc79fefdcb4676...|   21600|2010-04-04 00:00:00|     null|           Low|            17|
|01a041b0a5fbe79fb...|   59815|2010-12-15 00:00:00|     null| Not Specified|            14|
+--------------------+--------+-------------------+---------+--------------+--------------+
only showing top 4 rows



In [0]:
orders_df.filter("ship_date is not null").count()

Out[9]: 8109

In [0]:
orders_df = orders_df.withColumn("order_status",
                                            when((orders_df["ship_date"].isNull()) & (orders_df["order_date"] < current_date() - 4), "Cancelled") \
                                            .when((orders_df["ship_date"].isNull()) &  ((orders_df["order_date"] == current_date()) | (orders_df["order_date"] >= current_date() - 4)), "In Progress") \
                                            .otherwise("Completed")
                                            ) \
                     .withColumn("order_date", to_date("order_date")) \
                     .withColumn("order_id", col("order_id").cast("long"))


In [0]:
orders_df.show(5)

+--------------------+--------+----------+----------+--------------+--------------+------------+
|         customer_id|order_id|order_date| ship_date|order_priority|order_quantity|order_status|
+--------------------+--------+----------+----------+--------------+--------------+------------+
|001492ac094ba3c98...|   18374|2012-09-30|2012-10-02|           Low|            21|   Completed|
|001492ac094ba3c98...|   25314|2012-02-24|2012-02-26|          High|            43|   Completed|
|001492ac094ba3c98...|   31106|2012-02-16|2012-02-16|           Low|            37|   Completed|
|001492ac094ba3c98...|   31106|2012-02-16|2012-02-21|           Low|             7|   Completed|
|001492ac094ba3c98...|   31106|2012-02-16|2012-02-18|           Low|            45|   Completed|
+--------------------+--------+----------+----------+--------------+--------------+------------+
only showing top 5 rows



In [0]:
orders_df.filter("customer_id in ('07c41f7f8da2ed563ce0452e1d066605494817333732bafb29d6c596e6aeb39a','1212e8749bdc3800d2514d8ae53c10b8cbe3a9e4355de5db35f746c510c4603e','1312e8749bdc3800d2514d8ae53c10b8cbe3a9e4355de5db35f746c510c4603e','001492ac094ba3c986a45e7799a0409db64ed3c856e39cf16854012aa468fa01')").display()

customer_id,order_id,order_date,ship_date,order_priority,order_quantity,order_status
001492ac094ba3c986a45e7799a0409db64ed3c856e39cf16854012aa468fa01,18374,2012-09-30,2012-10-02,Low,21,Completed
001492ac094ba3c986a45e7799a0409db64ed3c856e39cf16854012aa468fa01,25314,2012-02-24,2012-02-26,High,43,Completed
001492ac094ba3c986a45e7799a0409db64ed3c856e39cf16854012aa468fa01,31106,2012-02-16,2012-02-16,Low,37,Completed
001492ac094ba3c986a45e7799a0409db64ed3c856e39cf16854012aa468fa01,31106,2012-02-16,2012-02-21,Low,7,Completed
001492ac094ba3c986a45e7799a0409db64ed3c856e39cf16854012aa468fa01,31106,2012-02-16,2012-02-18,Low,45,Completed
001492ac094ba3c986a45e7799a0409db64ed3c856e39cf16854012aa468fa01,31106,2012-02-16,2012-02-23,Low,43,Completed
001492ac094ba3c986a45e7799a0409db64ed3c856e39cf16854012aa468fa01,34663,2011-07-06,2011-07-08,Not Specified,38,Completed
001492ac094ba3c986a45e7799a0409db64ed3c856e39cf16854012aa468fa01,43493,2012-05-10,null,High,22,Cancelled
001492ac094ba3c986a45e7799a0409db64ed3c856e39cf16854012aa468fa01,43493,2012-05-10,2012-05-12,High,14,Completed
001492ac094ba3c986a45e7799a0409db64ed3c856e39cf16854012aa468fa01,43493,2012-05-10,2012-05-13,High,37,Completed


In [0]:
orders_df.groupBy('order_id').count().filter(col('count')>1).orderBy('count',ascending=False).show()

+--------+-----+
|order_id|count|
+--------+-----+
|   24132|    6|
|   43745|    6|
|   59781|    5|
|   48772|    5|
|   43875|    5|
|    1444|    5|
|   43488|    5|
|   13540|    5|
|   57253|    5|
|    8995|    5|
|   48452|    5|
|   29187|    5|
|   58470|    5|
|   12067|    5|
|   12261|    5|
|   27106|    5|
|   15109|    5|
|   33797|    5|
|   42528|    5|
|   58784|    5|
+--------+-----+
only showing top 20 rows



In [0]:
orders_df.filter("order_id = '48772'").show() #same order_id but products ordered will be different 

+--------------------+--------+----------+----------+--------------+--------------+------------+
|         customer_id|order_id|order_date| ship_date|order_priority|order_quantity|order_status|
+--------------------+--------+----------+----------+--------------+--------------+------------+
|dfff7e512bc9808c8...|   48772|2009-05-12|2009-05-14|        Medium|            44|   Completed|
|dfff7e512bc9808c8...|   48772|2009-05-12|2009-05-13|        Medium|            11|   Completed|
|dfff7e512bc9808c8...|   48772|2009-05-12|2009-05-13|        Medium|            18|   Completed|
|dfff7e512bc9808c8...|   48772|2009-05-12|2009-05-13|        Medium|            12|   Completed|
|dfff7e512bc9808c8...|   48772|2009-05-12|2009-05-13|        Medium|            10|   Completed|
+--------------------+--------+----------+----------+--------------+--------------+------------+



In [0]:
df.filter("order_id = '48772'").show() 

+------+--------+-------------------+--------------+--------------+-------+--------+-----------+------+----------+-------------+--------------+--------+------+----------------+----------------+--------------------+--------------------+-----------------+----------+--------------+------------+--------------------+------------------+--------------+-----------+
|row_id|order_id|         order_date|order_priority|order_quantity|  sales|discount|  ship_mode|profit|unit_price|shipping_cost| customer_name|province|region|customer_segment|product_category|product_sub_category|        product_name|product_container| ship_date|loyalty_points|loyalty_type|         customer_id|total_orders_count|customer_since|total_years|
+------+--------+-------------------+--------------+--------------+-------+--------+-----------+------+----------+-------------+--------------+--------+------+----------------+----------------+--------------------+--------------------+-----------------+----------+--------------+-

In [0]:
orders_df.groupBy('customer_id','order_id','order_date','ship_date','order_priority','order_quantity','order_status').count().filter(col('count')>1).orderBy('count',ascending=False).show(truncate=False)

+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+-----+
|customer_id                                                     |order_id|order_date|ship_date |order_priority|order_quantity|order_status|count|
+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+-----+
|07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd0|46916   |2011-10-03|2011-10-05|High          |40            |Completed   |3    |
|6380a2b9a6213ea50eacbe6393a7eaa6ce003677db42816672596d8add21632e|18368   |2012-11-30|2012-12-02|Not Specified |3             |Completed   |2    |
|64755dd6894b7317c422c86cf30c43c71cfcc4efcae2362acf98633e940242e8|43488   |2012-06-18|2012-06-19|Critical      |46            |Completed   |2    |
|3f0784e0bbdb7619091710f15923331f57e8dbf2d4e76e05866e7b5fa61e7974|11011   |2012-05-13|2012-05-14|High          |36    

In [0]:
orders_df.filter("order_id = '46916'").show(truncate=False) #same order_id but products are different

+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+
|customer_id                                                     |order_id|order_date|ship_date |order_priority|order_quantity|order_status|
+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+
|07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd0|46916   |2011-10-03|2011-10-05|High          |40            |Completed   |
|07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd0|46916   |2011-10-03|2011-10-05|High          |40            |Completed   |
|07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd0|46916   |2011-10-03|2011-10-05|High          |40            |Completed   |
+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+



In [0]:
df.filter("order_id = '46916'").show(truncate=False)

+------+--------+-------------------+--------------+--------------+-------+--------+-----------+------+----------+-------------+-------------+-------------+--------+----------------+----------------+--------------------+-----------------------------------------------------------------+-----------------+----------+--------------+------------+----------------------------------------------------------------+------------------+--------------+-----------+
|row_id|order_id|order_date         |order_priority|order_quantity|sales  |discount|ship_mode  |profit|unit_price|shipping_cost|customer_name|province     |region  |customer_segment|product_category|product_sub_category|product_name                                                     |product_container|ship_date |loyalty_points|loyalty_type|customer_id                                                     |total_orders_count|customer_since|total_years|
+------+--------+-------------------+--------------+--------------+-------+--------+------

In [0]:
#adding row_id column as its unique for each row
orders_df = df.select('row_id','customer_id', 'order_id', 'order_date', 'ship_date', 'order_priority', 'order_quantity')
orders_df = orders_df.withColumn("order_status",
                                            when((orders_df["ship_date"].isNull()) & (orders_df["order_date"] < current_date() - 4), "Cancelled") \
                                            .when((orders_df["ship_date"].isNull()) &  ((orders_df["order_date"] == current_date()) | (orders_df["order_date"] >= current_date() - 4)), "In Progress") \
                                            .otherwise("Completed")
                                            ) \
                    .withColumn("row_id", col("row_id").cast("long")) \
                     .withColumn("order_date", to_date("order_date")) \
                     .withColumn("order_id", col("order_id").cast("long"))

In [0]:
orders_df.filter("order_id = '46916'").show(truncate=False)

+------+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+
|row_id|customer_id                                                     |order_id|order_date|ship_date |order_priority|order_quantity|order_status|
+------+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+
|6592  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd0|46916   |2011-10-03|2011-10-05|High          |40            |Completed   |
|6593  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd0|46916   |2011-10-03|2011-10-05|High          |40            |Completed   |
|6594  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd0|46916   |2011-10-03|2011-10-05|High          |40            |Completed   |
+------+----------------------------------------------------------------+--------+----------+----------+--------

In [0]:
orders_df.orderBy("row_id",ascending=False).limit(1).show(truncate=False)

+------+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+
|row_id|customer_id                                                     |order_id|order_date|ship_date |order_priority|order_quantity|order_status|
+------+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+
|8399  |509e60618cef2ad50eda723bf33957832ecbce73b78cf2d067e6e1ea6ae450ea|59973   |2011-11-15|2011-11-19|Low           |30            |Completed   |
+------+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+



In [0]:
max_row_id = orders_df.agg({"row_id": "max"}).collect()[0][0]
#orders_df.select(max("row_id")).collect()[0][0]
print("max row id : ",max_row_id)

max row id :  8399


In [0]:
max_order_id = orders_df.agg({"order_id": "max"}).collect()[0][0]
#orders_df.select(max("row_id")).collect()[0][0]
print("max order id : ",max_order_id)

max order id :  59973


In [0]:
orders_df.printSchema()

root
 |-- row_id: long (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_id: long (nullable = true)
 |-- order_date: date (nullable = true)
 |-- ship_date: date (nullable = true)
 |-- order_priority: string (nullable = true)
 |-- order_quantity: long (nullable = true)
 |-- order_status: string (nullable = false)



In [0]:
orders_df.filter("order_id = '46916'").show(truncate=False)

+------+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+
|row_id|customer_id                                                     |order_id|order_date|ship_date |order_priority|order_quantity|order_status|
+------+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+
|6592  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd0|46916   |2011-10-03|2011-10-05|High          |40            |Completed   |
|6593  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd0|46916   |2011-10-03|2011-10-05|High          |40            |Completed   |
|6594  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd0|46916   |2011-10-03|2011-10-05|High          |40            |Completed   |
+------+----------------------------------------------------------------+--------+----------+----------+--------

In [0]:
orders_df.filter("customer_id = '1312e8749bdc3800d2514d8ae53c10b8cbe3a9e4355de5db35f746c510c4603e'").show(truncate=False)

+------+-----------+--------+----------+---------+--------------+--------------+------------+
|row_id|customer_id|order_id|order_date|ship_date|order_priority|order_quantity|order_status|
+------+-----------+--------+----------+---------+--------------+--------------+------------+
+------+-----------+--------+----------+---------+--------------+--------------+------------+



In [0]:
new_orders_data = [
    (8400, "07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd1", 59974, "High", 30),
    (8401, "1212e8749bdc3800d2514d8ae53c10b8cbe3a9e4355de5db35f746c510c4603f", 59975, "Low", 10),
    (8402, "1212e8749bdc3800d2514d8ae53c10b8cbe3a9e4355de5db35f746c510c4603f", 59975, "Low", 10)
]

# Create DataFrame for new order data
new_orders_df = spark.createDataFrame(new_orders_data, ["row_id", "customer_id", "order_id", "order_priority", "order_quantity" ])

In [0]:
#insert records
insert_records = new_orders_df.join(orders_df, ['row_id'], 'left_anti')

In [0]:
insert_records.show(truncate=False)

+------+----------------------------------------------------------------+--------+--------------+--------------+
|row_id|customer_id                                                     |order_id|order_priority|order_quantity|
+------+----------------------------------------------------------------+--------+--------------+--------------+
|8400  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd1|59974   |High          |30            |
|8401  |1212e8749bdc3800d2514d8ae53c10b8cbe3a9e4355de5db35f746c510c4603f|59975   |Low           |10            |
|8402  |1212e8749bdc3800d2514d8ae53c10b8cbe3a9e4355de5db35f746c510c4603f|59975   |Low           |10            |
+------+----------------------------------------------------------------+--------+--------------+--------------+



In [0]:
insert_records = insert_records.withColumn("order_date", current_date()-1) \
    .withColumn("ship_date", lit(None)) \
    .withColumn("order_status", lit("In Progress"))

In [0]:
insert_records = insert_records.select('row_id','customer_id', 'order_id', 'order_date', 'ship_date', 'order_priority', 'order_quantity','order_status')
insert_records.show(truncate=False)

+------+----------------------------------------------------------------+--------+----------+---------+--------------+--------------+------------+
|row_id|customer_id                                                     |order_id|order_date|ship_date|order_priority|order_quantity|order_status|
+------+----------------------------------------------------------------+--------+----------+---------+--------------+--------------+------------+
|8400  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd1|59974   |2024-05-03|null     |High          |30            |In Progress |
|8401  |1212e8749bdc3800d2514d8ae53c10b8cbe3a9e4355de5db35f746c510c4603f|59975   |2024-05-03|null     |Low           |10            |In Progress |
|8402  |1212e8749bdc3800d2514d8ae53c10b8cbe3a9e4355de5db35f746c510c4603f|59975   |2024-05-03|null     |Low           |10            |In Progress |
+------+----------------------------------------------------------------+--------+----------+---------+--------------+

In [0]:
full_run = 1 # 1 for full run and 0 for incremental run 

In [0]:
order_details_instance = "sales.orders_details" 

In [0]:
if full_run == 0:  #incremental run for updates -- only In progress records will be updated either to Completed or Cancelled
    results_df = spark.read.format("delta").table(order_details_instance).filter("order_status = 'In Progress'")

In [0]:
if full_run == 1:
    orders_df = orders_df.unionByName(insert_records)
    print(orders_df.count())
else:
    results_df = results_df.unionByName(insert_records)
    print(results_df.count())

8402


In [0]:
orders_df.filter("row_id in ('2533','8400') ").show(truncate=False)

+------+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+
|row_id|customer_id                                                     |order_id|order_date|ship_date |order_priority|order_quantity|order_status|
+------+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+
|2533  |001492ac094ba3c986a45e7799a0409db64ed3c856e39cf16854012aa468fa01|18374   |2012-09-30|2012-10-02|Low           |21            |Completed   |
|8400  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd1|59974   |2024-05-03|null      |High          |30            |In Progress |
+------+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+



In [0]:
if full_run == 1:
    max_row_id = orders_df.agg({"row_id": "max"}).collect()[0][0]
    #orders_df.select(max("row_id")).collect()[0][0]
    print("max row id : ",max_row_id)

    max_order_id = orders_df.agg({"order_id": "max"}).collect()[0][0]
    #orders_df.select(max("row_id")).collect()[0][0]
    print("max order id : ",max_order_id)

else:
    max_row_id = spark.sql("SELECT MAX(row_id) FROM sales.orders_details").collect()[0][0]
    #orders_df.select(max("row_id")).collect()[0][0]
    print("max row id : ",max_row_id)

    max_order_id = spark.sql("SELECT MAX(order_id) FROM sales.orders_details").collect()[0][0]
    #orders_df.select(max("row_id")).collect()[0][0]
    print("max order id : ",max_order_id)

max row id :  8402
max order id :  59975


In [0]:
schema = StructType([
    StructField("row_id", IntegerType(), True),  
    StructField("customer_id", StringType(), True), 
    StructField("order_id", IntegerType(), True), 
    StructField("order_date", StringType(), True), 
    StructField("ship_date", StringType(), True), 
    StructField("order_priority", StringType(), True), 
    StructField("order_quantity", IntegerType(), True) ,
    StructField("order_status", StringType(), True) 
])
new_orders_data = [
    (8400,"07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd1", 59974 ,   '2024-04-28',  '2024-04-30', "High", 30, None),
    (8401,"1212e8749bdc3800d2514d8ae53c10b8cbe3a9e4355de5db35f746c510c4603f", 59975 ,   '2024-04-30', None,         "Low", 10,None),
    (8402,"1212e8749bdc3800d2514d8ae53c10b8cbe3a9e4355de5db35f746c510c4603f", 59975 ,   '2024-04-30', None,         "Low", 10,None),
    (None,"07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd2", None,    None,        None,         "Medium",20,None),
    (None,"07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd3", None,    None,        None,         "Critical",60,None),
    (None,"07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd4", None,    None,        None,         "Low", 20,None),
    (None,"07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd5", None,    None,        None,         "High", 50,None),
    (None,"07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd6", None,    None,        None,         "High", 60,None),
    (None,"07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd6", None,    None,        None,         "Low", 20,None)
]

# Create DataFrame for new order data
new_orders_df = spark.createDataFrame(new_orders_data,schema).withColumn("order_date", to_date("order_date")).withColumn("ship_date", to_date("ship_date"))


In [0]:
new_orders_df.show(truncate=False)

+------+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+
|row_id|customer_id                                                     |order_id|order_date|ship_date |order_priority|order_quantity|order_status|
+------+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+
|8400  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd1|59974   |2024-04-28|2024-04-30|High          |30            |null        |
|8401  |1212e8749bdc3800d2514d8ae53c10b8cbe3a9e4355de5db35f746c510c4603f|59975   |2024-04-30|null      |Low           |10            |null        |
|8402  |1212e8749bdc3800d2514d8ae53c10b8cbe3a9e4355de5db35f746c510c4603f|59975   |2024-04-30|null      |Low           |10            |null        |
|null  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd2|null    |null      |null      |Medium  

In [0]:
if full_run == 1:
    insert_records = new_orders_df.join(orders_df, ['row_id'], 'left_anti')
else:
    insert_records = new_orders_df.join(results_df, ['row_id'], 'left_anti')

In [0]:
insert_records.show(truncate=False)

+------+----------------------------------------------------------------+--------+----------+---------+--------------+--------------+------------+
|row_id|customer_id                                                     |order_id|order_date|ship_date|order_priority|order_quantity|order_status|
+------+----------------------------------------------------------------+--------+----------+---------+--------------+--------------+------------+
|null  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd2|null    |null      |null     |Medium        |20            |null        |
|null  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd3|null    |null      |null     |Critical      |60            |null        |
|null  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd4|null    |null      |null     |Low           |20            |null        |
|null  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd5|null    |null      |null     |High          |

In [0]:
#row_id should be unique to each row whereas order_id shoud be same when customer_id is same for that particular day i,e, order_date
windowSpec = Window.orderBy(lit(1))
windowSpec1 = Window.orderBy("customer_id")

In [0]:
insert_records = insert_records.withColumn("row_id", max_row_id + row_number().over(windowSpec)) \
    .withColumn("customer_id_prev", lag("customer_id").over(windowSpec)) \
    .withColumn("order_date", current_date()-1) \
    .withColumn("ship_date", lit(None)) \
    .withColumn("order_status", lit("In Progress"))

In [0]:
insert_records = insert_records.withColumn("order_id_inc", when((col("customer_id_prev").isNull()) | (insert_records["customer_id"] != insert_records["customer_id_prev"]), 1).otherwise(0)) \
    .withColumn("order_id_offset", sum("order_id_inc").over(windowSpec1)) \
    .withColumn("order_id", max_order_id + col("order_id_offset")) 

In [0]:
insert_records.show(truncate=False)

+------+----------------------------------------------------------------+--------+----------+---------+--------------+--------------+------------+----------------------------------------------------------------+------------+---------------+
|row_id|customer_id                                                     |order_id|order_date|ship_date|order_priority|order_quantity|order_status|customer_id_prev                                                |order_id_inc|order_id_offset|
+------+----------------------------------------------------------------+--------+----------+---------+--------------+--------------+------------+----------------------------------------------------------------+------------+---------------+
|8403  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd2|59976   |2024-05-03|null     |Medium        |20            |In Progress |null                                                            |1           |1              |
|8404  |07de8c2693a047311aac3bb2e1a7

In [0]:
insert_records = insert_records.select('row_id','customer_id', 'order_id', 'order_date', 'ship_date', 'order_priority', 'order_quantity','order_status')

In [0]:
if full_run == 1:
    update_records = new_orders_df.join(orders_df, ['row_id'], 'inner').filter(orders_df["order_status"] == "In Progress").select(new_orders_df["row_id"],orders_df["customer_id"],orders_df["order_id"],new_orders_df["order_date"],new_orders_df["ship_date"],orders_df["order_priority"],orders_df["order_quantity"],orders_df["order_status"])
else:
    update_records = new_orders_df.join(results_df, ['row_id'], 'inner').select(new_orders_df["row_id"],results_df["customer_id"],results_df["order_id"],new_orders_df["order_date"],new_orders_df["ship_date"],results_df["order_priority"],results_df["order_quantity"],results_df["order_status"])

In [0]:
update_records = update_records.withColumn("order_status",
                                            when((update_records["ship_date"].isNull()) & (update_records["order_date"] < current_date() - 4), "Cancelled") \
                                            .when((update_records["ship_date"].isNull()) &  ((update_records["order_date"] == current_date()) | (update_records["order_date"] >= current_date() - 4)), "In Progress") \
                                            .otherwise("Completed")
                                            )

In [0]:
update_records.show(truncate=False)

+------+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+
|row_id|customer_id                                                     |order_id|order_date|ship_date |order_priority|order_quantity|order_status|
+------+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+
|8400  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd1|59974   |2024-04-28|2024-04-30|High          |30            |Completed   |
|8401  |1212e8749bdc3800d2514d8ae53c10b8cbe3a9e4355de5db35f746c510c4603f|59975   |2024-04-30|null      |Low           |10            |In Progress |
|8402  |1212e8749bdc3800d2514d8ae53c10b8cbe3a9e4355de5db35f746c510c4603f|59975   |2024-04-30|null      |Low           |10            |In Progress |
+------+----------------------------------------------------------------+--------+----------+----------+--------

In [0]:
if full_run == 0:
    results_df = results_df.join(update_records, on="row_id", how="left_anti") \
        .unionByName(insert_records) \
        .unionByName(update_records)
    results_df.count()
    results_df.createOrReplaceTempView("results_temp_vw")

In [0]:
if full_run == 1:
    orders_df = orders_df.join(update_records, on="row_id", how="left_anti") \
        .unionByName(insert_records) \
        .unionByName(update_records)
    orders_df.count()
else:
    spark.sql("""
        MERGE INTO sales.orders_details
        USING results_temp_vw
        ON sales.orders_details.row_id = results_temp_vw.row_id
        WHEN MATCHED THEN UPDATE SET *
        WHEN NOT MATCHED THEN INSERT *
    """)

In [0]:
orders_df.filter("order_status = 'In Progress'").show(truncate=False)

+------+----------------------------------------------------------------+--------+----------+---------+--------------+--------------+------------+
|row_id|customer_id                                                     |order_id|order_date|ship_date|order_priority|order_quantity|order_status|
+------+----------------------------------------------------------------+--------+----------+---------+--------------+--------------+------------+
|8403  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd2|59976   |2024-05-03|null     |Medium        |20            |In Progress |
|8404  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd3|59977   |2024-05-03|null     |Critical      |60            |In Progress |
|8405  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd4|59978   |2024-05-03|null     |Low           |20            |In Progress |
|8406  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd5|59979   |2024-05-03|null     |High          |

In [0]:
if full_run == 1:
    max_row_id = orders_df.agg({"row_id": "max"}).collect()[0][0]
    #orders_df.select(max("row_id")).collect()[0][0]
    print("max row id : ",max_row_id)

    max_order_id = orders_df.agg({"order_id": "max"}).collect()[0][0]
    #orders_df.select(max("row_id")).collect()[0][0]
    print("max order id : ",max_order_id)

else:
    max_row_id = spark.sql("SELECT MAX(row_id) FROM sales.orders_details").collect()[0][0]
    #orders_df.select(max("row_id")).collect()[0][0]
    print("max row id : ",max_row_id)

    max_order_id = spark.sql("SELECT MAX(order_id) FROM sales.orders_details").collect()[0][0]
    #orders_df.select(max("row_id")).collect()[0][0]
    print("max order id : ",max_order_id)

max row id :  8408
max order id :  59980


In [0]:
schema = StructType([
    StructField("row_id", IntegerType(), True),  
    StructField("customer_id", StringType(), True), 
    StructField("order_id", IntegerType(), True), 
    StructField("order_date", StringType(), True), 
    StructField("ship_date", StringType(), True), 
    StructField("order_priority", StringType(), True), 
    StructField("order_quantity", IntegerType(), True) ,
    StructField("order_status", StringType(), True) 
])
new_orders_data = [
    (8403,"07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd2", 59976,    '2024-04-30', '2024-05-01',  "Medium",20,None),
    (8404,"07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd3", 59977,    '2024-04-30', '2024-05-01',  "Critical",60,None),
    (8405,"07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd4", 59978,    '2024-04-30',  None,         "Low", 20,None),
    (8406,"07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd5", 59979,    '2024-04-30',  None,         "High", 50,None),
    (8407,"07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd6", 59980,    '2024-04-30',  None,         "High", 60,None),
    (8408,"07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd6", 59980,    '2024-04-30',  '2024-05-01', "Low", 20,None),
    (8409,"07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd7", 59981,    '2024-05-03',  None, "Low", 20,None),
    (8410,"07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd8", 59982,    '2024-05-03',  None, "Low", 20,None),
    (8411,"07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd9", 59983,    '2024-05-03',  '2024-05-04', "Low", 20,None),
    (8412,"07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd9", 59983,    '2024-05-03',  '2024-05-04', "Low", 20,None)
]

# Create DataFrame for new order data
new_orders_df = spark.createDataFrame(new_orders_data,schema).withColumn("order_date", to_date("order_date")).withColumn("ship_date", to_date("ship_date"))

In [0]:
new_orders_df.show(truncate=False)

+------+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+
|row_id|customer_id                                                     |order_id|order_date|ship_date |order_priority|order_quantity|order_status|
+------+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+
|8403  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd2|59976   |2024-04-30|2024-05-01|Medium        |20            |null        |
|8404  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd3|59977   |2024-04-30|2024-05-01|Critical      |60            |null        |
|8405  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd4|59978   |2024-04-30|null      |Low           |20            |null        |
|8406  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd5|59979   |2024-04-30|null      |High    

In [0]:
if full_run == 1:
    insert_records = new_orders_df.join(orders_df, ['row_id'], 'left_anti')
else:
    insert_records = new_orders_df.join(results_df, ['row_id'], 'left_anti')

In [0]:
insert_records.show(truncate=False)

+------+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+
|row_id|customer_id                                                     |order_id|order_date|ship_date |order_priority|order_quantity|order_status|
+------+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+
|8409  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd7|59981   |2024-05-03|null      |Low           |20            |null        |
|8410  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd8|59982   |2024-05-03|null      |Low           |20            |null        |
|8411  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd9|59983   |2024-05-03|2024-05-04|Low           |20            |null        |
|8412  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd9|59983   |2024-05-03|2024-05-04|Low     

In [0]:
windowSpec = Window.orderBy(lit(1))
windowSpec1 = Window.orderBy("customer_id")

In [0]:
insert_records = insert_records.withColumn("row_id", max_row_id + row_number().over(windowSpec)) \
    .withColumn("customer_id_prev", lag("customer_id").over(windowSpec)) \
    .withColumn("order_date", current_date()-1) \
    .withColumn("ship_date", lit(None)) \
    .withColumn("order_status", lit("In Progress"))

In [0]:
insert_records = insert_records.withColumn("order_id_inc", when((col("customer_id_prev").isNull()) | (insert_records["customer_id"] != insert_records["customer_id_prev"]), 1).otherwise(0)) \
    .withColumn("order_id_offset", sum("order_id_inc").over(windowSpec1)) \
    .withColumn("order_id", max_order_id + col("order_id_offset")) 

insert_records = insert_records.select('row_id','customer_id', 'order_id', 'order_date', 'ship_date', 'order_priority', 'order_quantity','order_status')

In [0]:
orders_df.filter("row_id = '8400'").show(truncate=False)

+------+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+
|row_id|customer_id                                                     |order_id|order_date|ship_date |order_priority|order_quantity|order_status|
+------+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+
|8400  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd1|59974   |2024-04-28|2024-04-30|High          |30            |Completed   |
+------+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+



In [0]:
orders_df.printSchema()

root
 |-- row_id: long (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_id: long (nullable = true)
 |-- order_date: date (nullable = true)
 |-- ship_date: date (nullable = true)
 |-- order_priority: string (nullable = true)
 |-- order_quantity: long (nullable = true)
 |-- order_status: string (nullable = false)



In [0]:
insert_records.show(truncate=False)

+------+----------------------------------------------------------------+--------+----------+---------+--------------+--------------+------------+
|row_id|customer_id                                                     |order_id|order_date|ship_date|order_priority|order_quantity|order_status|
+------+----------------------------------------------------------------+--------+----------+---------+--------------+--------------+------------+
|8409  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd7|59981   |2024-05-03|null     |Low           |20            |In Progress |
|8410  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd8|59982   |2024-05-03|null     |Low           |20            |In Progress |
|8411  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd9|59983   |2024-05-03|null     |Low           |20            |In Progress |
|8412  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd9|59983   |2024-05-03|null     |Low           |

In [0]:
if full_run == 1:
    update_records = new_orders_df.join(orders_df, ['row_id'], 'inner').filter(orders_df["order_status"] == "In Progress").select(new_orders_df["row_id"],orders_df["customer_id"],orders_df["order_id"],new_orders_df["order_date"],new_orders_df["ship_date"],orders_df["order_priority"],orders_df["order_quantity"],orders_df["order_status"])
else:
    update_records = new_orders_df.join(results_df, ['row_id'], 'inner').select(new_orders_df["row_id"],results_df["customer_id"],results_df["order_id"],new_orders_df["order_date"],new_orders_df["ship_date"],results_df["order_priority"],results_df["order_quantity"],results_df["order_status"])

In [0]:
update_records.show(truncate=False)

+------+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+
|row_id|customer_id                                                     |order_id|order_date|ship_date |order_priority|order_quantity|order_status|
+------+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+
|8403  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd2|59976   |2024-04-30|2024-05-01|Medium        |20            |In Progress |
|8404  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd3|59977   |2024-04-30|2024-05-01|Critical      |60            |In Progress |
|8405  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd4|59978   |2024-04-30|null      |Low           |20            |In Progress |
|8406  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd5|59979   |2024-04-30|null      |High    

In [0]:
update_records = update_records.withColumn("order_status",
                                            when((update_records["ship_date"].isNull()) & (update_records["order_date"] < current_date() - 4), "Cancelled") \
                                            .when((update_records["ship_date"].isNull()) &  ((update_records["order_date"] == current_date()) | (update_records["order_date"] >= current_date() - 4)), "In Progress") \
                                            .otherwise("Completed")
                                            )

In [0]:
update_records.show(truncate=False)

+------+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+
|row_id|customer_id                                                     |order_id|order_date|ship_date |order_priority|order_quantity|order_status|
+------+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+
|8403  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd2|59976   |2024-04-30|2024-05-01|Medium        |20            |Completed   |
|8404  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd3|59977   |2024-04-30|2024-05-01|Critical      |60            |Completed   |
|8405  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd4|59978   |2024-04-30|null      |Low           |20            |In Progress |
|8406  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd5|59979   |2024-04-30|null      |High    

In [0]:
if full_run == 0:
    results_df = results_df.join(update_records, on="row_id", how="left_anti") \
        .unionByName(insert_records) \
        .unionByName(update_records)
    results_df.count()
    results_df.createOrReplaceTempView("results_temp_vw")

In [0]:
if full_run == 1:
    orders_df = orders_df.join(update_records, on="row_id", how="left_anti") \
        .unionByName(insert_records) \
        .unionByName(update_records)
    orders_df.count()
else:
    spark.sql("""
        MERGE INTO sales.orders_details
        USING results_temp_vw
        ON sales.orders_details.row_id = results_temp_vw.row_id
        WHEN MATCHED THEN UPDATE SET *
        WHEN NOT MATCHED THEN INSERT *
    """)

In [0]:
if full_run == 1:
    orders_df.createOrReplaceTempView('orders_temp_vw')

    create_db_sql = """
        CREATE DATABASE IF NOT EXISTS sales
    """
    
    create_table_sql = """
        CREATE OR REPLACE TABLE sales.orders_details
        (
            row_id INT,
            customer_id STRING,
            order_id INT,
            order_date DATE,
            ship_date DATE,
            order_priority STRING,
            order_quantity INT,
            order_status STRING
        )
        USING DELTA
        LOCATION '/FileStore/tables/delta-table-merge/orders_details'
    """

In [0]:
if full_run == 1:
    spark.sql(create_db_sql)
    spark.sql(create_table_sql)

In [0]:
'''
%sql
select count(*) from sales.orders_details'''
spark.sql('''
          select count(*) from sales.orders_details
          ''').show(truncate=False)

+--------+
|count(1)|
+--------+
|8412    |
+--------+



#One Time Full Run for first run and whenever refresh data is required

In [0]:
if full_run == 1:
    # Truncate Delta table if full_run is 1
    spark.sql("TRUNCATE TABLE sales.orders_details")
    print("sales.orders_details table truncated.")
else:
    # Do nothing if full_run is not 1
    print("Skipping truncation. full_run is not set to 1.")

sales.orders_details table truncated.


In [0]:
if full_run == 1:
    # One time full load if full_run is 1
    spark.sql("INSERT INTO sales.orders_details SELECT * FROM orders_temp_vw")
    print("one time load successful.")
else:
    # Do nothing if full_run is not 1
    print("Skipping one time load. full_run is not set to 1.")

one time load successful.


In [0]:
spark.sql('''
          select * from sales.orders_details limit 5 
          ''').show(truncate=False)

+------+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+
|row_id|customer_id                                                     |order_id|order_date|ship_date |order_priority|order_quantity|order_status|
+------+----------------------------------------------------------------+--------+----------+----------+--------------+--------------+------------+
|5409  |03de3b10f1d1797fa42d5372db545dadd5aa777b823885979bdd09fa20e9660d|38466   |2011-08-17|2011-08-18|Critical      |22            |Completed   |
|7747  |0fb77b87360fcb496a32cb2673d291c1a83fd88c48d968a4b79d55a169e7144b|55459   |2009-04-25|2009-04-27|Critical      |5             |Completed   |
|29    |136d2dcc58235d6f3fecb6222bf3ae4c9bb00c635db1c1370edf2e86a903924e|194     |2012-04-04|2012-04-06|Medium        |6             |Completed   |
|2927  |1f07e41c5a310fda2c158f4a6760f4b7732a0f24c20db3466a661802b9b07bf9|21220   |2012-12-22|2012-12-25|Not Spec

In [0]:

spark.sql('''
          select * from sales.orders_details where customer_id= '07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd9'
          ''').show(truncate=False)

+------+----------------------------------------------------------------+--------+----------+---------+--------------+--------------+------------+
|row_id|customer_id                                                     |order_id|order_date|ship_date|order_priority|order_quantity|order_status|
+------+----------------------------------------------------------------+--------+----------+---------+--------------+--------------+------------+
|8411  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd9|59983   |2024-05-03|null     |Low           |20            |In Progress |
|8412  |07de8c2693a047311aac3bb2e1a7bf301d9e290b04431926048d92a40e3bbfd9|59983   |2024-05-03|null     |Low           |20            |In Progress |
+------+----------------------------------------------------------------+--------+----------+---------+--------------+--------------+------------+

